# NLP: Natural Language Processing com Pyspark

- Forçar a instalação do Pyspark para a versão 2.3.2. O motivo é que estava dando error com a versão mais recente.

In [13]:
#pip install --force-reinstall pyspark==2.3.2

In [14]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



In [15]:
# Tokenização
df = spark.createDataFrame([(1,'I really liked this movie'),
 (2,'I would recommend this movie to my friends'),
 (3,'movie was alright but acting was horrible'),
 (4,'I am never watching that movie ever again')],
 ['user_id','review'])

df.show(4,False) # False: mostra o texto completo

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



# Tokenização

- Temos que passar a coluna de entrada e nomear a coluna de saída após a tokenização. Usamos a função de transformação para aplicar tokenização à coluna de revisão.

In [16]:
from pyspark.ml.feature import Tokenizer

In [17]:
# Aplica a Função de Tokenização e cria um Output como coluna
tokenization = Tokenizer(inputCol='review',outputCol='tokens')
tokenized_df = tokenization.transform(df)
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



- Obtemos uma nova coluna denominada tokens que contém os tokens para cada frase.

# Stopwords Removal
    Processo de eliminar palavras que são irrelevantes ou agregam pouco as análises. A eliminação delas promove uma melhora das exigências computacionais. Palavras como:
        - 'this', 'the', 'to' , 'was', 'that',

In [18]:
from pyspark.ml.feature import StopWordsRemover

In [19]:
stopword_removal = StopWordsRemover(inputCol='tokens', outputCol='refined_tokens')

# Em seguida, passamos os tokens como a coluna de entrada e nomeamos a coluna de saída como tokens refinados.

refined_df = stopword_removal.transform(tokenized_df)

refined_df.select(['user_id','tokens','refined_tokens']).show(4,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



# Bag of Words
    - Consiste em representar os dados do texto em forma numérica para que sejam utilizados por Machine Learning ou qualquer outra análise. 
    - Count Vectorizer faz a contagem da palavra que aparece no documento específico. 
    
Desvantagem: A desvantagem de usar o método Count Vectorizer é que ele não considera as co-ocorrências de palavras em outros documentos. Em termos simples, as palavras que aparecem com mais frequência teriam um impacto maior no vetor de recursos. 

In [20]:
from pyspark.ml.feature import CountVectorizer

In [21]:
# Conta as palavras únicas para criação de um vetor númerico
count_vec = CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_df = count_vec.fit(refined_df).transform(refined_df)

cv_df.select(['user_id','refined_tokens','features']).show(4,False)

+-------+----------------------------------+--------------------------------+
|user_id|refined_tokens                    |features                        |
+-------+----------------------------------+--------------------------------+
|1      |[really, liked, movie]            |(11,[0,1,9],[1.0,1.0,1.0])      |
|2      |[recommend, movie, friends]       |(11,[0,5,10],[1.0,1.0,1.0])     |
|3      |[movie, alright, acting, horrible]|(11,[0,2,3,4],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(11,[0,6,7,8],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+--------------------------------+



- Como podemos observar, cada sentença é representada como um vetor denso. Ele mostra que o comprimento do vetor é 11 e a primeira sentença contém 3 valores nos índices 0, 4 e 9. 

In [22]:
# Para validar o vocabulário do vetorizador de contagem, podemos simplesmente usar a função de vocabulário.
count_vec.fit(refined_df).vocabulary

['movie',
 'acting',
 'horrible',
 'alright',
 'watching',
 'ever',
 'recommend',
 'never',
 'really',
 'liked',
 'friends']

# Term Frequency – Inverse Document Frequency (TF-IDF)

É outra abordagem para converter dados de texto em formato numérico.

Este método tenta normalizar a frequência de ocorrência de palavras com base em outros documentos. A ideia é dar mais peso à palavra se ela aparecer mais vezes no mesmo documento, mas penalizar se ela aparecer mais vezes em outros documentos também.

Isso indica que uma palavra é comum em todo o corpus e não é tão importante quanto indica sua frequência no documento atual. 

    - Term Frequency: Pontuação com base na frequência da palavra no documento atual. 

    - Inverse Document Frequency: Pontuação com base na frequência de documentos que contém a palavra atual.

In [23]:
from pyspark.ml.feature import HashingTF,IDF

In [24]:
# Usado no mesmo dataframe df refinado.
hashing_vec = HashingTF(inputCol='refined_tokens',outputCol='tf_features')

hashing_df = hashing_vec.transform(refined_df)

hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

+-------+----------------------------------+-------------------------------------------------------+
|user_id|refined_tokens                    |tf_features                                            |
+-------+----------------------------------+-------------------------------------------------------+
|1      |[really, liked, movie]            |(262144,[99172,210223,229264],[1.0,1.0,1.0])           |
|2      |[recommend, movie, friends]       |(262144,[68228,130047,210223],[1.0,1.0,1.0])           |
|3      |[movie, alright, acting, horrible]|(262144,[95685,171118,210223,236263],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(262144,[63139,113673,203802,210223],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+-------------------------------------------------------+



In [25]:
tf_idf_vec = IDF(inputCol='tf_features',outputCol='tf_idf_features')

tf_idf_df = tf_idf_vec.fit(hashing_df).transform(hashing_df)

tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

+-------+----------------------------------------------------------------------------------------------------+
|user_id|tf_idf_features                                                                                     |
+-------+----------------------------------------------------------------------------------------------------+
|1      |(262144,[99172,210223,229264],[0.9162907318741551,0.0,0.9162907318741551])                          |
|2      |(262144,[68228,130047,210223],[0.9162907318741551,0.9162907318741551,0.0])                          |
|3      |(262144,[95685,171118,210223,236263],[0.9162907318741551,0.9162907318741551,0.0,0.9162907318741551])|
|4      |(262144,[63139,113673,203802,210223],[0.9162907318741551,0.9162907318741551,0.9162907318741551,0.0])|
+-------+----------------------------------------------------------------------------------------------------+

